# 8.0 Tries

## 8.1 Implement autocomplete

### Problem Statement
Implement an autocomplete feature for a search engine.  In other words given a string prefix, return all of the words in the index which start with the prefix.

#### Examples
Index: [`the`, `their`, `there`, `was`, `waste`]

Search Term: `the` <br/>
Returns: `the`, `their`, `there`

Search Term: `wa` <br/>
Returns: `was`, `waste`

Search Term: `wasp` <br/>
Returns: <>

In [1]:
from collections import defaultdict, deque, namedtuple
import unittest


class Trie(object):
    """Trie implemented using nested python dicts."""
    
    # Non-alphabet character used to identify end of word.
    EOW = '.'

    def __init__(self):
        self.index = defaultdict(dict)

    def add_word(self, word):
        node = self.index
        for ind, c in enumerate(word):
            if c in node:
                node = node[c]
            else:
                node[c] = {}
                node = node[c]
        if len(word) > 0:
            node[self.EOW] = ''

    def search(self, prefix):
        """Search for words starting with prefix."""
        assert not(len(prefix) < 1), 'invalid: len(word) < 1'

        matches, node = [], self.index

        # Find the furthest matching node in the tree.
        for c in prefix:
            if c not in node:
                return matches  # No matching prefix.
            node = node[c]

        # Perform a depth first traversal of the subtree
        # accumulating all of the words matching prefix.
        stack = deque([(node, '')])
        while len(stack) > 0:
            node, suffix = stack.popleft()
            for c in node.keys():
                if c == self.EOW:
                    matches.append(prefix + suffix)
                else:
                    stack.appendleft((node[c], suffix+c))
        
        return matches


class AutocompleteTest(unittest.TestCase):
    
    def setUp(self):
        self.words1 = ['t','the','their','there','was','waste']

    def test_autocomplete(self):
        case = namedtuple('case', ['words','prefix','expected'])
        cases = [
            case(self.words1, 't', ['t','the','their','there']),
            case(self.words1, 'th', ['the','their','there']),
            case(self.words1, 'the', ['the','their','there']),
            case(self.words1, 'them', []),
            case(self.words1, 'thei', ['their']),
            case(self.words1, 'their', ['their']),
            case(self.words1, 'theirs', []),
            case(self.words1, 'ther', ['there']),
            case(self.words1, 'there', ['there']),
            case(self.words1, 'therein', []),
            case(self.words1, 'w', ['was', 'waste']),
            case(self.words1, 'wb', []),
            case(self.words1, 'wa', ['was', 'waste']),
            case(self.words1, 'was', ['was', 'waste']),
            case(self.words1, 'wae', []),
            case(self.words1, 'wast', ['waste']),
            case(self.words1, 'waste', ['waste']),
            case(self.words1, 'wastes', []),
            case(self.words1, 'x', []),
        ]
        for c in cases:
            trie = Trie()
            for word in c.words:
                trie.add_word(word)
            rcv = trie.search(c.prefix)
            self.assertEqual(sorted(rcv), sorted(c.expected))


unittest.main(AutocompleteTest(), argv=[''], verbosity=2, exit=False)

test_autocomplete (__main__.AutocompleteTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK
